In [3]:
# import numpy, pandas, sklearn for k-means and pca


import numpy as np
import pandas as pd
import sklearn
import pickle
import matplotlib.pyplot as plt
import umap
import cv2

import keras
import collections

from scipy import ndimage
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
#from sklearn.datasets import load_digits

In [4]:
# import the data from pickle
src = "./data/LSWMD.pkl"
data=pd.read_pickle(src)


# clean the label data as I would be using it for getting a subset
def clean(x):
    if(len(x) == 0):
        return ""
    else:
        return x[0][0]

def replacecolumn(column):
    newcolumn = []
    for entry in column:
        if(len(entry)==0):
            entry = ""
        else:
            entry = entry[0][0]
        newcolumn.append(entry)
    return newcolumn

collabel = replacecolumn(data["trianTestLabel"])
colfailuretype = replacecolumn(data["failureType"])

data["trianTestLabel"] = collabel
data["failureType"] = colfailuretype

# clean unlabeled data
print(data.shape)
data = data[data["failureType"] != ""]
print(data.shape)

(811457, 6)
(172950, 6)


In [5]:
# use the training data as using the whole data set is costly and results in kernel crash.
subdata = data[data['trianTestLabel'] == "Training"] #data cleaning of [["Training"]]-> "Training" needed

#check only defective ones as the defects are in one cluster
#subdata = subdata[data['failureType'] != "none"] #data cleaning of [["Training"]]-> "Training" needed

# fetch the wafer map and failure type
waferdata = subdata["waferMap"]
labelwaferdata = subdata["failureType"]

lb_make = LabelEncoder()
labelwaferdata = lb_make.fit_transform(labelwaferdata)
print(labelwaferdata[0])

# the idea is to have all the circular defects be close to the normal by making any defect at the edge to be rotated to be at the bottom
def rotationmax(wafer):
    rotationlist = []
    rotangle = 60 # having a degree of not 90 results in more empty space which can be resolved with 
    steps = 360//rotangle
    for i in range(steps):
        rotated = ndimage.rotate(wafer, rotangle*i, reshape=False)
        rotationlist.append(rotated)
    
    
    intensitylist = []
    for image in rotationlist:
        #3*(len(a)//4):len(a),:
        intensitysum = sum(sum(image[1*(size[1]//2):size[1],:]))
        intensitylist.append(intensitysum)
    
    idx = np.argmax(intensitylist)
    
    #print(len(rotationlist),idx,steps,intensitylist)

    newwafer = rotationlist[idx]
    
    return newwafer
    
# resize the wafer maps to be uniform

size = (51,51)
thresh = 1.5

newwaferdata = []
for idx,wafer in enumerate(waferdata):
    wafer = cv2.medianBlur(wafer, 5)
    wafer = cv2.medianBlur(wafer, 3)
    wafer = cv2.resize(wafer, dsize=size, interpolation=cv2.INTER_CUBIC)

    #wafer = cv2.medianBlur(wafer, 3)
    #wafer = rotationmax(wafer)
    wafer = cv2.threshold(wafer, thresh, 255, cv2.THRESH_BINARY)[1]    
    #print(wafer.shape)
    #wafer = np.array(wafer).reshape(size[0]*size[1])
    newwaferdata.append(wafer)

newwaferdata = np.array(newwaferdata)
print(newwaferdata.shape,newwaferdata[0].shape)
   
# rotations need to happen to every image as the label data isn't known.

8
(54355, 51, 51) (51, 51)


In [23]:
# the difference here is that instead of just having a flat vector, try using an autoencoder
from keras import layers

length = newwaferdata.shape[0]


flatnewwaferdata = newwaferdata.reshape((length,size[0]*size[1]))
inputshape = flatnewwaferdata[0].shape
print(inputshape)

encoding_dim = 32
input_img = keras.Input(shape=inputshape)
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(inputshape[0], activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)


# encoder and decoder
encoder = keras.Model(input_img, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

# compile
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

print(autoencoder)

(2601,)


In [ ]:
autoencoder.fit(flatnewwaferdata, flatnewwaferdata,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(flatnewwaferdata, flatnewwaferdata))


Train on 54355 samples, validate on 54355 samples
Epoch 1/50
54355/54355 [==============================] - 7s 123us/step - loss: -410041.6172 - val_loss: -1391904.4536
Epoch 2/50
54355/54355 [==============================] - 6s 118us/step - loss: -3548488.4436 - val_loss: -6310831.6805
Epoch 3/50
54355/54355 [==============================] - 6s 116us/step - loss: -10133375.9245 - val_loss: -14499324.4957
Epoch 4/50
54272/54355 [============================>.] - ETA: 0s - loss: -19856037.6981